In [13]:
import pandas as pd

!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [14]:
from pyspark import SparkContext
from pyspark.sql import SparkSession 
from pyspark.ml.feature import StringIndexer,VectorAssembler

spark = SparkSession.builder\
            .appName("Spark Dataframe Intro")\
            .getOrCreate()

In [15]:
tags = spark.read.option("header","true").option("inferSchema","false").csv('/content/tags.csv')
ratings = spark.read.option("header","true").option("inferSchema","true").csv('/content/ratings.csv')
movies = spark.read.option("header","true").option("inferSchema","false").csv('/content/movies.csv')

In [16]:
tags.show()

+------+-------+-----------------+----------+
|userId|movieId|              tag| timestamp|
+------+-------+-----------------+----------+
|     2|  60756|            funny|1445714994|
|     2|  60756|  Highly quotable|1445714996|
|     2|  60756|     will ferrell|1445714992|
|     2|  89774|     Boxing story|1445715207|
|     2|  89774|              MMA|1445715200|
|     2|  89774|        Tom Hardy|1445715205|
|     2| 106782|            drugs|1445715054|
|     2| 106782|Leonardo DiCaprio|1445715051|
|     2| 106782|  Martin Scorsese|1445715056|
|     7|  48516|     way too long|1169687325|
|    18|    431|        Al Pacino|1462138765|
|    18|    431|         gangster|1462138749|
|    18|    431|            mafia|1462138755|
|    18|   1221|        Al Pacino|1461699306|
|    18|   1221|            Mafia|1461699303|
|    18|   5995|        holocaust|1455735472|
|    18|   5995|       true story|1455735479|
|    18|  44665|     twist ending|1456948283|
|    18|  52604|  Anthony Hopkins|

In [17]:
ratings=ratings.withColumnRenamed("movieId","movieId2")
ratings.show()

+------+--------+------+---------+
|userId|movieId2|rating|timestamp|
+------+--------+------+---------+
|     1|       1|   4.0|964982703|
|     1|       3|   4.0|964981247|
|     1|       6|   4.0|964982224|
|     1|      47|   5.0|964983815|
|     1|      50|   5.0|964982931|
|     1|      70|   3.0|964982400|
|     1|     101|   5.0|964980868|
|     1|     110|   4.0|964982176|
|     1|     151|   5.0|964984041|
|     1|     157|   5.0|964984100|
|     1|     163|   5.0|964983650|
|     1|     216|   5.0|964981208|
|     1|     223|   3.0|964980985|
|     1|     231|   5.0|964981179|
|     1|     235|   4.0|964980908|
|     1|     260|   5.0|964981680|
|     1|     296|   3.0|964982967|
|     1|     316|   3.0|964982310|
|     1|     333|   5.0|964981179|
|     1|     349|   4.0|964982563|
+------+--------+------+---------+
only showing top 20 rows



In [18]:
join_table = tags.join(ratings,'userId','inner')
#join_table.show()

In [19]:
join_table = join_table.drop('timestamp')
join_table = join_table.drop('movieId2')

In [20]:
indxr = StringIndexer(inputCol="tag",outputCol="tags")
join_table = indxr.fit(join_table).transform(join_table)

In [21]:
join_table = join_table.drop('tag')

In [22]:
join_table.show()

+------+-------+------+-----+
|userId|movieId|rating| tags|
+------+-------+------+-----+
|     2| 106782|   3.0|295.0|
|     2| 106782|   3.0| 22.0|
|     2| 106782|   3.0|296.0|
|     2|  89774|   3.0| 78.0|
|     2|  89774|   3.0|294.0|
|     2|  89774|   3.0|292.0|
|     2|  60756|   3.0| 23.0|
|     2|  60756|   3.0|293.0|
|     2|  60756|   3.0|  0.0|
|     2| 106782|   4.0|295.0|
|     2| 106782|   4.0| 22.0|
|     2| 106782|   4.0|296.0|
|     2|  89774|   4.0| 78.0|
|     2|  89774|   4.0|294.0|
|     2|  89774|   4.0|292.0|
|     2|  60756|   4.0| 23.0|
|     2|  60756|   4.0|293.0|
|     2|  60756|   4.0|  0.0|
|     2| 106782|   4.5|295.0|
|     2| 106782|   4.5| 22.0|
+------+-------+------+-----+
only showing top 20 rows



In [23]:
join_table=join_table.drop('tags')

In [42]:
table_pivot = join_table.groupBy("userId").pivot("movieId").avg().fillna(0)

In [43]:
table_pivot.show()

+------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+--------------

In [44]:
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from pyspark.ml.feature import StringIndexer,VectorAssembler
from pyspark.ml.classification import RandomForestClassifier


vec = VectorAssembler(inputCols=table_pivot.columns[1:],outputCol="features")
table_pivot_vec = vec.transform(table_pivot)
table_pivot_vec = table_pivot_vec.select("userId","features")


In [45]:
table_pivot_vec.createOrReplaceTempView("cartesian")

In [46]:
table_pivot_vec = spark.sql("select a.userId as userıd1,a.features as features1,b.features as features2 ,b.userID AS userID2 from cartesian a cross join cartesian b")

In [47]:
import pyspark.sql.functions as F
from pyspark.sql.types import FloatType
from scipy.spatial import distance
distance_udf = F.udf(lambda x, y: float(distance.euclidean(x, y)), FloatType())
distance_df = table_pivot_vec.withColumn('distance', distance_udf(F.col('features2'),F.col('features1')))

In [48]:
distance_df.show(10)

+-------+--------------------+--------------------+-------+---------+
|userıd1|           features1|           features2|userID2| distance|
+-------+--------------------+--------------------+-------+---------+
|      7|(83,[55],[3.23026...|(83,[55],[3.23026...|      7|      0.0|
|      7|(83,[55],[3.23026...|(83,[13,21,49,50,...|     18|10.389087|
|      7|(83,[55],[3.23026...|(83,[5],[4.261904...|     49|  5.34775|
|      7|(83,[55],[3.23026...|(83,[34],[3.63099...|     63| 4.859911|
|      7|(83,[55],[3.23026...|[4.08196721311475...|     62|34.060886|
|      7|(83,[55],[3.23026...|(83,[11,12,22],[3...|     21|6.5062685|
|      7|(83,[55],[3.23026...|(83,[34],[3.08403...|     76| 4.466079|
|      7|(83,[55],[3.23026...|(83,[2,67,80],[3....|      2|7.5631504|
|     18|(83,[13,21,49,50,...|(83,[55],[3.23026...|      7|10.389087|
|     18|(83,[13,21,49,50,...|(83,[13,21,49,50,...|     18|      0.0|
+-------+--------------------+--------------------+-------+---------+
only showing top 10 

In [ ]:
distance_df.createOrReplaceTempView("a")

In [ ]:
spark.sql("""select * from (select distinct userıd1,  DENSE_RANK() OVER (partition by userıd1 ORDER BY userıd1,distance asc)
  similatiry_degree,userID2 as similiar_user_of_users, distance from a where distance > 0 ORDER BY userıd1,distance ) where similatiry_degree <= 10 """).show(100)

+-------+-----------------+----------------------+---------+
|userıd1|similatiry_degree|similiar_user_of_users| distance|
+-------+-----------------+----------------------+---------+
|     18|                1|                    49|11852.096|
|     18|                2|                     7|11859.223|
|     18|                3|                     2|11863.981|
|     18|                4|                    76|11871.767|
|     18|                5|                    63|12011.374|
|     18|                6|                    21|12366.065|
|     18|                7|                    62| 75840.22|
|      2|                1|                    49| 652.7396|
|      2|                2|                     7|771.39984|
|      2|                3|                    76|944.84534|
|      2|                4|                    63| 2055.967|
|      2|                5|                    21|3587.9602|
|      2|                6|                    18|11863.981|
|      2|               